In [23]:
# Import the important packages
import pandas as pd

pd.set_option('display.max_columns', None)

In [24]:
# Load the raw data
gamesdf = pd.read_csv('../../data/games.csv')

In [25]:
gamesdf.columns

Index(['Unnamed: 0', 'away.teamStats.teamSkaterStats.blocked',
       'away.teamStats.teamSkaterStats.faceOffWinPercentage',
       'away.teamStats.teamSkaterStats.giveaways',
       'away.teamStats.teamSkaterStats.goals',
       'away.teamStats.teamSkaterStats.hits',
       'away.teamStats.teamSkaterStats.pim',
       'away.teamStats.teamSkaterStats.powerPlayGoals',
       'away.teamStats.teamSkaterStats.powerPlayOpportunities',
       'away.teamStats.teamSkaterStats.powerPlayPercentage',
       'away.teamStats.teamSkaterStats.shots',
       'away.teamStats.teamSkaterStats.takeaways', 'content.link', 'date',
       'gameDate', 'gameID', 'gamePk', 'gameType',
       'home.teamStats.teamSkaterStats.blocked',
       'home.teamStats.teamSkaterStats.faceOffWinPercentage',
       'home.teamStats.teamSkaterStats.giveaways',
       'home.teamStats.teamSkaterStats.goals',
       'home.teamStats.teamSkaterStats.hits',
       'home.teamStats.teamSkaterStats.pim',
       'home.teamStats.teamSkate

In [26]:
def create_lag(df, p, sortList, partList, lagList):
    """
    Cette fonction prend comme intrant un df, une liste de variables pour lesquelles il faut
    creer des valeurs retardees, le nombre de retard, les variables sur lesquelles il faut sort,
    les variables de partition et la liste de varibles pour lesquelles on veut des valeurs retardees.
    La fonction retourne un df augmente des variables retardees
    """
    df = df.sort_values(by=sortList)
    
    for feature in lagList:
        for i in range(p):
            df['{}.L{}'.format(feature,i+1)] = df.groupby(partList)[feature].shift(i+1)
    
    return df

In [27]:
# List des features

features5L = ['away.teamStats.teamSkaterStats.blocked','away.teamStats.teamSkaterStats.faceOffWinPercentage',
              'away.teamStats.teamSkaterStats.giveaways','away.teamStats.teamSkaterStats.goals',
              'away.teamStats.teamSkaterStats.hits','away.teamStats.teamSkaterStats.pim',
              'away.teamStats.teamSkaterStats.powerPlayGoals','away.teamStats.teamSkaterStats.powerPlayOpportunities',
              'away.teamStats.teamSkaterStats.powerPlayPercentage','away.teamStats.teamSkaterStats.shots',
              'away.teamStats.teamSkaterStats.takeaways','home.teamStats.teamSkaterStats.blocked',
              'home.teamStats.teamSkaterStats.faceOffWinPercentage','home.teamStats.teamSkaterStats.giveaways',
              'home.teamStats.teamSkaterStats.goals','home.teamStats.teamSkaterStats.hits',
              'home.teamStats.teamSkaterStats.pim','home.teamStats.teamSkaterStats.powerPlayGoals',
              'home.teamStats.teamSkaterStats.powerPlayOpportunities','home.teamStats.teamSkaterStats.powerPlayPercentage',
              'home.teamStats.teamSkaterStats.shots','home.teamStats.teamSkaterStats.takeaways']

features1L = ['teams.away.leagueRecord.losses','teams.away.leagueRecord.ot',
              'teams.away.leagueRecord.wins','teams.home.leagueRecord.losses',
              'teams.home.leagueRecord.ot','teams.home.leagueRecord.wins']

#Variables date, saisons et equipes
state_time = ['teams.home.team.name', 'season', 'gamePk']


In [28]:
# Creation de la variable a predire -- home win
gamesdf['game.homewin'] = (gamesdf['teams.home.score'] - gamesdf['teams.away.score']> 0).astype('int')

In [29]:
gamesdf_ = gamesdf[['game.homewin'] + features5L + features1L + state_time]

In [30]:
gamesdf_ = create_lag(df=gamesdf_,
                      p=1,
                      sortList=['teams.home.team.name', 'season', 'gamePk'],
                      partList=['teams.home.team.name', 'season'],
                      lagList=features1L)

In [31]:
gamesdf_ = create_lag(df=gamesdf_,
                      p=10,
                      sortList=['teams.home.team.name', 'season', 'gamePk'],
                      partList=['teams.home.team.name', 'season'],
                      lagList=features5L)

In [32]:
# Maintenant, on drop les variables qu'on utilisera pas pour predire....
gamesdf_final = gamesdf_.drop(columns= state_time + features1L + features5L)


In [33]:
# Export en csv..
gamesdf_final.to_csv('../../data/model_input_v2.csv')